In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as clr
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.decomposition import PCA

In [19]:
#Veri setinin eklenmesi
url="https://raw.githubusercontent.com/emrullahozkilinc/machine-learning/master/dataset/default%20of%20credit%20card%20clients.xls"
df = pd.read_excel(url, header=1)

Veri seti hakkında bilgi için:
https://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients

In [3]:
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,1,20000,2,2,1,24,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,0,2,2682,1725,2682,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,0,0,29239,14027,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,0,0,46990,48233,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,0,0,8617,5670,35835,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [4]:
df.columns

Index(['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0',
       'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'default payment next month'],
      dtype='object')

In [5]:
df['PAY_0'].unique()

array([ 2, -1,  0, -2,  1,  3,  4,  8,  7,  5,  6])

In [20]:
#Hedef kolon ismi uzun olduğundan ismini DEFAULT olarak değiştirilmesi.
df.rename({'default payment next month' : 'DEFAULT'}, axis="columns", inplace=True)

In [21]:
#ID kolonunun işe yaramadığı için silinmesi.
df.drop("ID", axis='columns', inplace=True)
df.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,DEFAULT
0,20000,2,2,1,24,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0,1
1,120000,2,2,2,26,-1,2,0,0,0,2,2682,1725,2682,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,90000,2,2,2,34,0,0,0,0,0,0,29239,14027,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,50000,2,2,1,37,0,0,0,0,0,0,46990,48233,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,50000,1,2,1,57,-1,0,-1,0,0,0,8617,5670,35835,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [30]:
#EDUCATION ve MARRIAGE kolonlarındaki missing valueların hesaplanması
len(df.loc[(df['EDUCATION']==0) | (df['MARRIAGE']==0)])

68

In [31]:
#İki kolondada missing value olmayan tüm satırların alınması.
df_no_missing = df.loc[(df['EDUCATION']!=0) & (df['MARRIAGE']!=0)]
len(df_no_missing)

29932

In [35]:
#Default=0 ve Default=1 olanların ayrılması.
df_default = df_no_missing[df_no_missing['DEFAULT']==1]
df_no_default = df_no_missing[df_no_missing['DEFAULT']==0]

print(len(df_default))
print(len(df_no_default))

6631
23301


In [34]:
#Her biri için 1000'er rasgele örnek alınması
df_default_resampled = resample(df_default, replace=False, random_state=42, n_samples=1000)
df_no_default_resampled = resample(df_no_default, replace=False, random_state=42, n_samples=1000)

print(len(df_default_resampled))
print(len(df_no_default_resampled))

1000
1000


In [37]:
#Alınan örneklerin tek bir dataframe'de birleştirilmesi
df_resampled = pd.concat([df_default_resampled,df_no_default_resampled])
len(df_resampled)

2000

In [39]:
#test edilecek verilerden hedef kolonun silinmesi.
X = df_resampled.drop(['DEFAULT'],axis='columns').copy()
X.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
19982,300000,2,1,1,47,3,2,2,2,2,2,5000,5000,5000,5000,5000,5000,0,0,0,0,0,0
19350,80000,2,2,2,36,2,0,0,-2,-2,-2,19671,20650,0,0,0,0,1700,0,0,0,0,0
17057,30000,2,3,2,22,2,2,0,0,0,0,29793,29008,29047,29507,11609,11711,0,1687,1147,524,400,666
26996,80000,1,1,2,34,2,2,2,2,2,2,61231,62423,63827,64682,65614,67007,2800,3000,2500,2600,2600,2600
23621,210000,2,3,1,44,-2,-2,-2,-2,-2,-2,11771,13462,17706,0,5646,14793,13462,17706,0,5646,14793,7376


In [40]:
#Hedef kolonun değişkene atanması
y = df_resampled.copy(['DEFAULT'])

In [43]:
#Modelin train edilebilmesi için one-hot-encoding yapılması.
encode_columns = ['SEX','EDUCATION', 'MARRIAGE','PAY_0', 'PAY_2',
       'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6']
X_encoded = pd.get_dummies(X, columns=encode_columns)

In [47]:
#Verinin train-test'e bölünmesi ve scale edilmesi.
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, random_state=42)
X_train_scaled=scale(X_train)
X_test_scaled=scale(X_test)